# Supplemental Notebook A: Gene Annotation Correction in GSM iYLI647
This Jupyter notebook is dedicated to correcting minor errors in gene annotations within the iYLI647 Genome-Scale Model (GSM) of Yarrowia lipolytica, ensuring that the gene mapping aligns accurately with transcriptomic data.

Objective
The primary goal is to refine the GSM by:
- Identifying and correcting duplicate gene entries in the GSM.
- Removing genes not present in the transcriptomics dataset.
- Fixing typographical errors in gene IDs.
- Analyzing and adjusting the biomass composition in the model for consistency.

In [1]:
import pandas as pd
import cobra

### Load genome scale model

In [2]:
model = cobra.io.json.load_json_model("../genome_scale_models/iYLI647_corr.json")
model

Name,model
Memory address,11007d960
Number of metabolites,1121
Number of reactions,1348
Number of genes,646
Number of groups,0
Objective expression,1.0*biomass_C - 1.0*biomass_C_reverse_c1d5c
Compartments,"c, e, m, n, x, r, g, v"


### Fix reaction that has a duplicate gene

In [3]:
# fix Diphosphoglyceromutase gene_reaction_rule
print(model.reactions.DPGM.gene_reaction_rule)

# remove duplicate genes
model.reactions.DPGM.gene_reaction_rule = 'YALI0B02728g or YALI0D09229g'

display(model.reactions.DPGM)

YALI0B02728g or (YALI0D09229g and YALI0D09229g)


Reaction identifier,DPGM
Name,Diphosphoglyceromutase
Memory address,0x2af67e4a0
Stoichiometry,13dpg[c] <=> 23dpg[c] + h[c] 3_Phospho_D_glyceroyl_phosphate <=> 2_3_Disphospho_D_glycerate + H
GPR,YALI0B02728g or YALI0D09229g
Lower bound,-1000.0
Upper bound,1000.0


### Remove genes that are not in the transcriptomics data set

In [4]:
# fix cytochrome c oxidase mitochondrial gene_reaction_rule
print(model.reactions.CYOOm.gene_reaction_rule)

# remove misanotated gene (YALI0F04)
model.reactions.CYOOm.gene_reaction_rule = 'YALI0E10144g and YALI0E12628g and YALI0E16709g and YALI0E19723g and YALI0F03201g and YALI0F03567g and YALI0F04103g'

display(model.reactions.CYOOm)

YALI0E10144g and YALI0E12628g and YALI0E16709g and YALI0E19723g and YALI0F03201g and YALI0F03567g and YALI0F04103g and YALI0F04


Reaction identifier,CYOOm
Name,cytochrome c oxidase mitochondrial
Memory address,0x2af7dcc70
Stoichiometry,4.0 focytc[m] + 6.0 h[m] + o2[m] --> 4.0 ficytc[m] + 2.0 h2o[m] + 6.0 h[c] 4.0 Ferrocytochrome_c + 6.0 H + O2 --> 4.0 Ferricytochrome_c + 2.0 H2O + 6.0 H
GPR,YALI0E10144g and YALI0E12628g and YALI0E16709g and YALI0E19723g and YALI0F03201g and YALI0F03567g...
Lower bound,0.0
Upper bound,1000.0


In [5]:
# ubiquinol 6 cytochrome c reductase gene_reaction_rule
print(model.reactions.CYOR_u6m.gene_reaction_rule)

# remove misanotated gene (YALI0F24)
model.reactions.CYOR_u6m.gene_reaction_rule = 'YALI0A02915g and YALI0A17468g and YALI0B01540g and YALI0C12210g and YALI0E34037g and YALI0F01771g and YALI0F08613g'

display(model.reactions.CYOR_u6m)

YALI0A02915g and YALI0A17468g and YALI0B01540g and YALI0C12210g and YALI0E34037g and YALI0F01771g and YALI0F08613g and YALI0F24


Reaction identifier,CYOR_u6m
Name,ubiquinol 6 cytochrome c reductase
Memory address,0x2af7de5c0
Stoichiometry,2.0 ficytc[m] + 1.5 h[m] + q6h2[m] --> 2.0 focytc[m] + 1.5 h[c] + q6[m] 2.0 Ferricytochrome_c + 1.5 H + Ubiquinol_6 --> 2.0 Ferrocytochrome_c + 1.5 H + Ubiquinone_6
GPR,YALI0A02915g and YALI0A17468g and YALI0B01540g and YALI0C12210g and YALI0E34037g and YALI0F01771g...
Lower bound,0.0
Upper bound,1000.0


### Fix gene ids that have typos

In [6]:
# fix typo in hexadecanoate transport gene_reaction_rule
print(model.reactions.HCAt.gene_reaction_rule)

# fix typo (YALIB02544g)
model.reactions.HCAt.gene_reaction_rule = 'YALI0C20265g or YALI0B02544g'

display(model.reactions.HCAt)

YALI0C20265g or YALIB02544g


Reaction identifier,HCAt
Name,Hexanoate n C60 transport in via uniport YL
Memory address,0x2afba1960
Stoichiometry,hdca[e] --> hdca[c] Hexadecanoate__n_C160_ --> Hexadecanoate__n_C160_
GPR,YALI0C20265g or YALI0B02544g
Lower bound,0.0
Upper bound,1000.0


In [7]:
# fix typo in decanoate transport gene_reaction_rule
print(model.reactions.dca_t.gene_reaction_rule)

# fix typo (YALIB02544g)
model.reactions.dca_t.gene_reaction_rule = 'YALI0C20265g or YALI0B02544g'

display(model.reactions.dca_t)

YALI0C20265g or YALIB02544g


Reaction identifier,dca_t
Name,decanoate transport in via uniport
Memory address,0x2afba2230
Stoichiometry,dca[e] --> dca[c] Decanoate__n_C100_ --> Decanoate__n_C100_
GPR,YALI0C20265g or YALI0B02544g
Lower bound,0.0
Upper bound,1000.0


In [8]:
# fix typo in Acetyl-CoA:acetoacetyl-CoA transferase gene_reaction_rule
print(model.reactions.ACACCT.gene_reaction_rule)

# fix typo (YALI00F26587g)
model.reactions.ACACCT.gene_reaction_rule = 'YALI0F26587g'

display(model.reactions.ACACCT)

YALI00F26587g


Reaction identifier,ACACCT
Name,Acetyl-CoA:acetoacetyl-CoA transferase
Memory address,0x2afbf97e0
Stoichiometry,acac[c] + accoa[c] --> aacoa[c] + ac[c] Acetoacetate + Acetyl_CoA --> Acetoacetyl_CoA + Acetate
GPR,YALI0F26587g
Lower bound,0.0
Upper bound,1000.0


### Analyze biomass composition

In [9]:
biomass_reaction_string = model.reactions.get_by_id('biomass_N').reaction
print(biomass_reaction_string)
reactant_string = biomass_reaction_string.split(' --> ')[0]
product_string = biomass_reaction_string.split(' --> ')[1]
reactant_components = reactant_string.split(' + ')
product_components = product_string.split(' + ')

# print(reactant_string)

component_details = []

# loop over reactant and product components similar to for component in join(reactant_components, product_components):

for component in reactant_components:
    [coefficient, metabolite_id] = component.split(' ')
    metabolite = model.metabolites.get_by_id(metabolite_id)
    component_details.append({
        'metabolite_id': metabolite_id,
        'metabolite name': metabolite.name,
        'metabolite formula': metabolite.formula,
        'molar mass (mg/mmol)': metabolite.formula_weight,
        'GSM coefficient': coefficient,
        'GSM mass (mg)': metabolite.formula_weight * float(coefficient),
    })

for component in product_components:
    if component == 'biomass[c]':
        coefficient = '1'
        metabolite_id = 'biomass[c]'
    else:
        [coefficient, metabolite_id] = component.split(' ')

    metabolite = model.metabolites.get_by_id(metabolite_id)
    component_details.append({
        'metabolite_id': metabolite_id,
        'metabolite name': metabolite.name,
        'metabolite formula': metabolite.formula,
        'molar mass (mg/mmol)': metabolite.formula_weight,
        'GSM coefficient': coefficient,
        'GSM mass (mg)': metabolite.formula_weight * float(coefficient),
    })
    print(coefficient, metabolite_id)

# make a dataframe
biomass_reaction_df = pd.DataFrame(component_details)

# save as csv
biomass_reaction_df.to_csv('../results/biomass_reaction_components.csv', index=False)

0.257113988 13BDglcn[c] + 0.09496 ala_L[c] + 0.001432042 amp[c] + 0.018065337 arg_L[c] + 0.026287726 asn_L[c] + 0.026287726 asp_L[c] + 20.31 atp[c] + 0.236682322 chitin[c] + 0.001478116 cmp[c] + 0.000519556 cys_L[c] + 0.016513908 damp[c] + 0.017106666 dcmp[c] + 0.015134716 dgmp[c] + 0.016975477 dtmp[c] + 0.060650767 ergst[c] + 0.032192196 gln_L[c] + 0.032192196 glu_L[c] + 0.082618568 gly[c] + 0.001315238 gmp[c] + 20.31 h2o[c] + 0.007031076 his_L[c] + 0.013733758 ile_L[c] + 0.02888825 leu_L[c] + 0.031527805 lys_L[c] + 0.064278269 mannan[c] + 0.002859543 met_L[c] + 8.468939e-05 pa_SC[c] + 0.00057788053 pc_SC[c] + 0.00057788053 pe_SC[c] + 0.012013414 phe_L[c] + 0.028688078 pro_L[c] + 0.00010959803 ps_SC[c] + 0.00012553956 ptd1ino_SC[c] + 0.046748124 ser_L[c] + 0.02 so4[c] + 0.031790673 thr_L[c] + 0.001869797 tre[c] + 0.00038910587 triglyc_SC[c] + 0.000287711 trp_L[c] + 0.006237863 tyr_L[c] + 0.001533699 ump[c] + 0.025946327 val_L[c] + 0.003484382 zymst[c] --> 20.31 adp[c] + 20.31 h[c] + 2

In [10]:
biomass_reaction_df 

,metabolite_id,metabolite name,metabolite formula,molar mass (mg/mmol),GSM coefficient,GSM mass (mg)
0,13BDglcn[c],1_3_beta_D_Glucan,C6H10O5,162.140600,0.257113988,41.688616
1,ala_L[c],L_Alanine,C3H7NO2,89.093180,0.09496,8.460288
2,amp[c],AMP,C10H12N5O7P,345.205341,0.001432042,0.494349
3,arg_L[c],L_Arginine,C6H15N4O2,175.208900,0.018065337,3.165208
4,asn_L[c],L_Asparagine,C4H8N2O3,132.117920,0.026287726,3.473080
5,asp_L[c],L_Aspartate,C4H6NO4,132.094740,0.026287726,3.472470
6,atp[c],ATP,C10H12N5O13P3,503.149263,20.31,10218.961532
7,chitin[c],Chitin__monomer_,C8H13NO5,203.192520,0.236682322,48.092077
8,cmp[c],CMP,C9H12N3O8P,321.180641,0.001478116,0.474742
9,cys_L[c],L_Cysteine,C3H7NO2S,121.158180,0.000519556,0.062948


### Save the updated model

In [11]:
# save the updated model
# cobra.io.save_json_model(model, "../genome_scale_models/iYLI647_corr_2.json")
# model